In [1]:
import json
import openai
import datetime
import obsidiantools.api as otools
from pathlib import Path
import os

In [26]:
wkd = Path(os.getcwd()).parent.parent.parent
vault = otools.Vault(wkd).connect().gather()

corpus = []
for k, v in vault.readable_text_index.items():
    corpus.append(v)


In [27]:
out_file_name = f"./fine_tune_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl"

for e in corpus:
    with open(out_file_name, "a+") as outfile:
        json.dump({
            "prompt": f"",
            "completion": f" {e} END",
        }, outfile)
        outfile.write('\n')

In [ ]:
!head -n3 $out_file_name | jq .

In [32]:
!echo "execute: openai tools fine_tunes.prepare_data -f $(pwd)/$out_file_name"

execute: openai tools fine_tunes.prepare_data -f /Users/louisbeaumont/Documents/brain/.obsidian/plugins/obsidian-ava/./fine_tune_17_09_2022.jsonl


In [33]:
from sklearn.model_selection import train_test_split
import pandas as pd
train, test = train_test_split(
    open(f"{out_file_name.replace('.jsonl', '_prepared.jsonl')}").read().splitlines(), test_size=0.1)
with open(f"{out_file_name.replace('.jsonl', '')}_prepared_train.jsonl", "w") as outfile:
    outfile.write("\n".join(train))
with open(f"{out_file_name.replace('.jsonl', '')}_prepared_test.jsonl", "w") as outfile:
    outfile.write("\n".join(test))

In [ ]:
!head -n3 {out_file_name.replace('.jsonl', '')}_prepared_train.jsonl | jq .

In [31]:
from openai.wandb_logger import WandbLogger
import wandb
import re
values = open(".env", "r").read()
wandb_key = re.findall(r"WANDB_KEY=\"(.*)\"", values)[0]
wandb.login(key=wandb_key, relogin=True)
openai.api_key = re.findall(r"OPENAI_API_KEY=\"(.*)\"", values)[0]
openai.organization = re.findall(r"OPENAI_ORGANIZATION=\"(.*)\"", values)[0]

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/louisbeaumont/.netrc


In [35]:
train_file = openai.File.create(
  file=open(f"{out_file_name.replace('.jsonl', '')}_prepared_train.jsonl", "rb"),
  purpose="fine-tune"
)
valid_file = openai.File.create(
  file=open(f"{out_file_name.replace('.jsonl', '')}_prepared_test.jsonl", "rb"),
  purpose="fine-tune"
)
ft = openai.FineTune.create(
    training_file=train_file["id"],
    validation_file=valid_file["id"],
    model="curie",
)

In [40]:
WandbLogger.sync(
    id=ft["id"],
    project="obsidian-openai",
    tags=["generation"],
)

wandb: Currently logged in as: louis030195. Use `wandb login --relogin` to force relogin


wandb: WARNING Serializing object of type str that is 209420 bytes


elapsed_examples,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
elapsed_tokens,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
training_loss,▅▄▆▆▅▅▄▇▅▄▄▄▅▆▆▅▆▄▆█▆▅▆▄▆█▄▅▃▄▄▅▁▃▄▂▄▄▂▁
training_sequence_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
training_token_accuracy,▄▄▄▂▄▅▂▁▃▅▅▅▄▂▂▃▃▃▄▂▄▅▄▄▄▂▃▂▄▆▅▄█▆▃▆▄▄▆█
validation_loss,▆▆▄▃▆▆▆▆▃▃▅▄▆▄▁▃▄▅▅▄▄▆▅▆▄▃▇▆▄▄▆▃▄▄▄▄▅▄█▅
validation_sequence_accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_token_accuracy,▁▃▃▂▁▃▃▂▆▆▃▂▂▂█▄▂▂▁▃▃▃▂▄▂▄▂▂▄▃▂▅▅▃▃▅▃▃▁▂
elapsed_examples,6664.0
elapsed_tokens,1966264.0
fine_tuned_model,curie:ft-personal-20...


'🎉 wandb sync completed successfully'

In [16]:
import re
import openai
import datetime
values = open(".env", "r").read()
openai.api_key = re.findall(r"OPENAI_API_KEY=\"(.*)\"", values)[0]
openai.organization = re.findall(r"OPENAI_ORGANIZATION=\"(.*)\"", values)[0]
model = "curie:ft-personal-2022-09-17-07-49-44"
outputs = []
async def comp():
    response = openai.Completion.create(
      model=model,
      prompt="",
      temperature=0.7,
      max_tokens=256,
      top_p=1,
      frequency_penalty=0,
      presence_penalty=0,
      stop=["END"]
    )
    outputs.append(response["choices"][0]["text"])
import asyncio
# generate 100 samples into a file
await asyncio.gather(*[comp() for _ in range(10)])
  
with open(f"./{model}_{datetime.date.today().strftime('%d_%m_%Y')}.jsonl", "w") as outfile:
  outfile.write(("\n"+("-"*5)+"\n").join(outputs))